In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv(
    "../../../Datasets/RegExpPURE/PURE_train.csv",
    usecols=["Requirement", "Req/Not Req"],
)

test_df = pd.read_csv(
    "../../../Datasets/RegExpPURE/PURE_test.csv",
    usecols=["Requirement", "Req/Not Req"],
)

valid_df = pd.read_csv(
    "../../../Datasets/RegExpPURE/PURE_valid.csv",
    usecols=["Requirement", "Req/Not Req"],
)

In [ ]:
train_X = train_df["Requirement"].values
train_y = train_df["Req/Not Req"].map({"Req": 1, "Not_Req": 0}).values

test_X = test_df["Requirement"].values
test_y = test_df["Req/Not Req"].map({"Req": 1, "Not_Req": 0}).values

valid_X = valid_df["Requirement"].values
valid_y = valid_df["Req/Not Req"].map({"Req": 1, "Not_Req": 0}).values

## Roberta For Sequence Classification

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
import torch

### Custom Dataset Class

In [ ]:
# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, X, y, tokenizer, max_length):
        self.X = X
        self.y = y
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        text = self.X[idx]
        label = self.y[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label": torch.tensor(label),
        }

In [ ]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base").to(device)

# Define hyperparameters
batch_size = 16
max_length = 128
learning_rate = 2e-5
num_epochs = 10

# Create dataloaders for training, validation, and testing
train_dataset = CustomDataset(train_X, train_y, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = CustomDataset(valid_X, valid_y, tokenizer, max_length)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

test_dataset = CustomDataset(test_X, test_y, tokenizer, max_length)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

## Training Loop

In [ ]:
total_validation_accuracy = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)

    # Validation loop
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in valid_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted_labels = torch.max(logits, dim=1)

            total_correct += (predicted_labels == labels).sum().item()
            total_samples += labels.size(0)

    accuracy = total_correct / total_samples
    total_validation_accuracy += accuracy

    print(
        f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f} - Validation Accuracy: {accuracy:.4f}"
    )

print(f"Average Validation Accuracy: {total_validation_accuracy / num_epochs:.4f}")

### Testing Loop

In [ ]:
# Testing loop
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted_labels = torch.max(logits, dim=1)

        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples

print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
# Save the model

model.save_pretrained("../../../Models/Roberta_for_Sequence_Classification")